# Image Support
- Up to 100 images across all messages in a single request
- Max size 5MB
- Max height/width - per single image and multiple images; can be included in base64 encoding or URL to image
- Tokens per image = (width px x height px)/750
- PROMPT ENGINEERING - KEY - It is very important to optimise Prompts. Use same techniques: Guidelines, Analysis Steps, One-Shot/Multi-Shot examples

# PDF & CITATIONS
- PDF Also Supported
- Citations Supported (Argument within create - Message)
- In Response: Citation block in message - cited_text, document_index, document_title, start_page_number, end_page_number
- User assured authoritative answer (and not Claude interpretation)
- Citations can be used with PDF, Text...etc
- Can create UIs with references to documents summarised by Claude

In [20]:
import base64

from anthropic import Anthropic
# Load env variables and create client
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"

In [21]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
        messages,
        system=None,
        temperature=1.0,
        stop_sequences=[],
        tools=None,
        thinking=False,
        thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [22]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure

4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [23]:
# # IMAGE PROCESSING
# messages = []
#
# with open("./images/prop2.png", "rb") as f:
#     image_bytes = base64.standard_b64encode(f.read()).decode("utf8")
#
# add_user_message(messages, [
#     #Image Block
#     {
#         "type": "image",
#         "source": {
#             "type": "base64",
#             "media_type": "image/png",
#             "data": image_bytes
#         }
#     },
#     #Text Block
#     {
#         "type": "text",
#         "text": prompt
#     }
# ])
#
# chat(messages)

In [24]:
# PDF PROCESSING WITH CITATIONS
messages = []

with open("./earth.pdf", "rb") as f:
    image_bytes = base64.standard_b64encode(f.read()).decode("utf8")

add_user_message(messages, [
    #Image Block
    {
        "type": "document",
        "source": {
            "type": "base64",
            "media_type": "application/pdf",
            "data": image_bytes
        },
        "title": "earth.pdf",
        "citations": {"enabled": True}
    },
    #Text Block
    {
        "type": "text",
        "text": "Summarise document in 1 sentence"
    }
])

chat(messages)

Message(id='msg_01TPKxtQJrhhzyX2HQ5JEwD8', content=[TextBlock(citations=[CitationPageLocation(cited_text="Earth\r\nThe Blue Marble, Apollo 17, December 1972\r\nDesignations\r\nAlternative\r\nnames\r\nThe world · The globe ·\r\nTerra · Tellus · Gaia ·\r\nMother Earth · Sol III\r\nAdjectives Earthly · Terrestrial · Terran\r\n· Tellurian\r\nSymbol and\r\nOrbital characteristics\r\nEpoch J2000\r\n[n 1]\r\nAphelion 152 097 597 km\r\nPerihelion 147 098 450 km\r\n[n 2]\r\nSemi-major axis 149 598 023 km\r\n[1]\r\nEccentricity 0.016 7086\r\n[1]\r\nOrbital period\r\n(sidereal)\r\n365.256 363 004 d\r\n[2]\r\n(1.000 017 420 96 aj)\r\nAverage orbital\r\nspeed\r\n29.7827 km/s\r\n[3]\r\nMean anomaly 358.617°\r\nInclination 7.155° – Sun's equator;\r\nEarth\r\nEarth is the third planet from the Sun and the only\r\nastronomical object known to harbor life. This is\r\nenabled by Earth being an ocean world, the only one in\r\nthe Solar System sustaining liquid surface water. Almost\r\nall of Earth's water